In [ ]:
from collections import defaultdict

class Grammar:
    def __init__(self, tokens, rules, ssymbol):
        self.tokens  = tokens
        self.rules   = rules
        self.ssymbol = ssymbol


    def remove_not_productive(self):
        def sentence_productive(sentence, productive_nonterminals, nonterminals):
            return all( map(lambda symbol: symbol not in nonterminals or symbol in productive_nonterminals, sentence))

        nonterminals = self.rules.keys()
        productives = set()
        
        symbol_added = True
        while (symbol_added):
            symbol_added = False
            for nonterminal, sentences in self.rules.items():
                if nonterminal not in productives:
                    for sentence in sentences:
                        if (sentence_productive(sentence, productives, nonterminals)):
                            symbol_added = True
                            productives.add(nonterminal)
                            break
        
        new_rules = defaultdict(list)
        for nonterminal, sentences in self.rules.items():
            if nonterminal in productives:
                for sentence in sentences:
                    if (sentence_productive(sentence, productives, nonterminals)):
                        new_rules[nonterminal].append(sentence)

        self.rules = new_rules

    def remove_unreachable(self):
        def get_nonterminals(sentence):
            return { nonterminal for nonterminal in sentence if isinstance(nonterminal,str) }

        reachable = set(self.ssymbol)

        symbol_added = True
        while (symbol_added):
            symbol_added = False
            for nonterminal, sentences in self.rules.items():
                if nonterminal in reachable:
                    for sentence in sentences:
                        symbol_added = get_nonterminals(sentence) - reachable
                        reachable|=get_nonterminals(sentence)
        
        new_rules = defaultdict(list)
        for nonterminal, sentences in self.rules.items():
            if nonterminal in reachable:
                new_rules[nonterminal] = sentences                

        self.rules = new_rules
                

    def remove_useless(self):
        self.remove_not_productive()
        self.remove_unreachable()


    def vanishing(self):
        def sentence_vanishing(sentence, vanishing_nonterminals):
            return all( map(lambda symbol: symbol == (0, 'e') or symbol in vanishing_nonterminals,sentence ))

        vanishing_nonterminals = set()

        symbol_added = True
        while (symbol_added):
            symbol_added = False
            for nonterminal, sentences in self.rules.items():
                if (nonterminal not in vanishing_nonterminals):
                    for sentence in sentences:
                        if (sentence_vanishing(sentence, vanishing_nonterminals)):
                            symbol_added=True
                            vanishing_nonterminals.add(nonterminal)
                            break
            
        return vanishing_nonterminals


def main():
    grammar = Grammar(
        {
            (0,'e'),
            (1,'a'),
            (2,'b'),
            (3,'c'),
            (4,'d'),
            (5,'f'),
        },
        {
            'A' : [ 
                [(1,'a'), (2,'b')],
                ['B', 'F']
            ],
            'B' : [ 
                [(0,'e')],
                ['B', 'F']
            ],
            'C' : [
                ['F']
            ],
            'F' : [
                ['F']
            ],
            'S' : [
                ['A']
            ]
        },
        'S'
    )
    print(grammar.vanishing())
    grammar.remove_not_productive()
    grammar.remove_unreachable()
    print(grammar.rules)


if __name__ == "__main__":
    main()